In [1]:
# Importing library
import numpy as np
import serial
import array
import time
import binascii
import sys
import numpy
from matplotlib import pyplot as plt
import struct


class HL_emulator:
    def __init__(self, port="", speed_rate=921600):

        self.port = port
        self.speed_rate = speed_rate

    def serial_start(self):
        # If you have problems please check that library pyserial is installed and lib serial is not installed
        # Also you need USB-COM driver cp2102
        # Probably you don't know the number of serial com port in you PC, then you need to push win+R and write command "devmgmt.msc". After that find 'com_port' tab and silicon lab devices.
        ser = serial.Serial(
            self.port,
            self.speed_rate,
            timeout=1,
            xonxoff=False,
            rtscts=False,
            dsrdtr=False,
        )
        self.ser = ser
        ser.setRTS(False)
        ser.setDTR(False)
        print("Port connected")

    def serial_close(self):
        self.ser.close()
        print("Port disconnected")

    def get_serial(self):
        return self.ser

    def serial_is_open(self):
        if self.ser.isOpen()==True:
            print("Serial is opening")
        else:
            print("Serial is close")
        return self.ser.isOpen()
        

class STM_API:

    def __init__(self, ser):
        self.ser = ser

    def CRC_Calc(self, packet: bytearray):
        crc = 0
        for ik in range(2, packet[2] + 2):
            crc += packet[ik]
        crc = ~crc
        crc &= 0xFF
        return bytearray([crc])

    def Send_Packet(self, cmd: bytes, data: bytearray, data_length: bytes):
        packet = bytearray([0xFF, 0xFF, data_length + 2, cmd]) + data
        packet += self.CRC_Calc(packet)
        self.ser.flushInput()
        self.ser.write(packet)
        return 0

    def robot_set_coord(self, x: float, y: float, phi: float):
        cmd = 0x06
        data = bytearray(self.struct.pack("f", x))
        data += bytearray(self.struct.pack("f", y))
        data += bytearray(self.struct.pack("f", phi))
        data_length = 12
        self.Send_Packet(cmd, data, data_length)

    def robot_set_speed(self, dx: float, dy: float, dphi: float):
        cmd = 0x04
        data = bytearray(struct.pack("f", dx))
        data += bytearray(struct.pack("f", dy))
        data += bytearray(struct.pack("f", dphi))
        data_length = 12
        self.Send_Packet(cmd, data, data_length)

    def robot_get_speed(self):
        cmd = 0x05
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
        rx_packet = self.ser.read(17)
        speed = struct.unpack("f", rx_packet[4:8])
        speed += struct.unpack("f", rx_packet[8:12])
        speed += struct.unpack("f", rx_packet[12:16])
        return speed

    def robot_get_coord(self):
        cmd = 0x07
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
        rx_packet = self.ser.read(17)
        coord = self.struct.unpack("f", rx_packet[4:8])
        coord += self.struct.unpack("f", rx_packet[8:12])
        coord += self.struct.unpack("f", rx_packet[12:16])
        return coord

    def get_angle(self,ID):
        cmd = 0x08
        data = bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd,data,data_length)

        rx_packet = self.ser.read(7)
        angle = struct.unpack('h', rx_packet[4:6])
        print(angle)
    
    def set_angle(self, ID, angle):
        cmd = 0x03
        data = bytearray([ID,angle&0xFF,(angle>>8)&0xFF])
        data_length = 3
        self.Send_Packet(cmd,data,data_length)
        time.sleep(0.01)
    
    def grub_pile(self, side):
        cmd = 0x09
        

        data = bytearray([side])
        data_length = 1

        self.Send_Packet(cmd,data,data_length)
    
    def release_pile(self,side):
        cmd = 0x0A
        

        data = bytearray([side])
        data_length = 1

        self.Send_Packet(cmd,data,data_length)
    
    def get_grub_status(self,side):
        cmd = 0x0B
        data = bytearray(struct.pack('i', side))
        data_length = 4
        self.Send_Packet(cmd,data,data_length)
        rx_packet = self.ser.read(6)
        status = rx_packet[4]
        return status

    def get_release_status(self,side):
        cmd = 0x0D
        data = bytearray(struct.pack('i', side))
        data_length = 4
        self.Send_Packet(cmd,data,data_length)
        rx_packet = self.ser.read(6)
        status = rx_packet[4]
        return status

    def set_wheel_mode(self, ID):
        cmd = 0x0F
        data = bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)

    def set_joint_mode(self, ID):
        cmd = 0x10
        data = bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)
    
    def set_speed(self, ID, speed):
        cmd = 0x11
        # data = bytearray([ID, *speed.to_bytes(2, 'little')])
        data =bytearray([ID])
        data += bytearray(struct.pack('h', speed))
        data_length = 3
        self.Send_Packet(cmd, data, data_length)

    def step_angle(self, ID, angle):
        cmd = 0x13
        data =bytearray([ID])
        data += bytearray(struct.pack('i', angle))
        data_length = 5
        self.Send_Packet(cmd, data, data_length)

    def stop_stepper(self, ID):
        cmd = 0x14
        data =bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)

    def set_pwm(self, ID, speed):
        cmd = 0x12
        data = bytearray([ID, *speed.to_bytes(2, 'little')])
        data_length = 3
        self.Send_Packet(cmd, data, data_length)

    def calibrate(self, ID):
        self.set_pwm(ID, 100)
        time.sleep(2)
        self.set_pwm(ID, 200)
        time.sleep(3)
        self.set_pwm(ID, 100)
        
    def robot_set_speed_e(self, dx: float, dy: float, dphi: float):
        cmd = 0x15
        data = bytearray(struct.pack("f", dx))
        data += bytearray(struct.pack("f", dy))
        data += bytearray(struct.pack("f", dphi))
        data_length = 12
        self.Send_Packet(cmd, data, data_length)

    def motor_flag(self):
        cmd = 0x16
        data= bytearray(struct.pack("?", 1))
        data_length = 1
        self.Send_Packet(cmd,data,data_length)

    def motor_on(self):
        cmd = 0x17
        data= bytearray(struct.pack("?", 1))
        data_length = 1
        self.Send_Packet(cmd,data,data_length)

    def motor_off(self):
        cmd = 0x17
        data= bytearray(struct.pack("?", 0))
        data_length = 0
        self.Send_Packet(cmd,data,data_length)



In [2]:
vova_em='/dev/cu.SLAB_USBtoUART'
ser_em_desk="COM4"
ser_em_laptop="COM30"

emulator=HL_emulator(port=ser_em_desk)
emulator.serial_start()
emulator.serial_is_open()
ST=STM_API(ser=emulator.get_serial())

Port connected
Serial is opening


In [88]:
ST.motor_on()

In [28]:
ST.motor_flag()

In [596]:
ST.motor_off()

In [20]:
emulator.serial_close()

Port disconnected


In [25]:
ST.step_angle(0, -2000)

In [16]:
ST.step_angle(0, 6400)

In [85]:
for i in range (0,10):
    ST.step_angle(0, 3200)
    time.sleep(2)

SerialException: WriteFile failed (PermissionError(13, 'Устройство не опознает команду.', None, 22))

In [69]:
ST.stop_stepper(3)

In [87]:
ST.robot_set_speed_e(0.8, 0, 0)
time.sleep(1)
now = time.time_ns()
while(1):
    speed = ST.robot_get_speed()[0]
    if abs(speed) < 0.01:
        print(speed)
        print(time.time_ns() - now)
        break
    

AttributeError: 'STM_API' object has no attribute 'struct'

In [238]:
ST.robot_set_speed_e(0.1, 0.1, 0.1)

In [239]:
ST.robot_set_speed(0, 0, 0)

In [214]:
ST.robot_get_speed()

(-0.008603401482105255, -0.004967175889760256, 0.092412568628788)